DATA643- Project 2
======
Prashanth Padebetttu, Parshu Rath, Adjare Windokun, Xingjia Wu
----------
<b> Summer 2016 <br>
Instructor: Andrew Catlin

<b> Load 100K movie data as pandas dataframe:

In [2]:
# Required Python Packages
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
import csv
import pandas as pd
import numpy as np
from scipy import spatial

In [3]:
#Load data into a pandas dataframe
# Movielens data with 100,000 records
url = 'https://raw.githubusercontent.com/ppadebettu/CUNY/IS-607-Data-Acquisition-and-Management/IS643/Project_2/movielens.csv'
df = pd.read_csv(url, sep = "," , header = 0, na_values='NaN')


In [4]:
df.head()

,userid,age,gender,occupation,zipcode,itemid,rating,timestamp,movieid,movietitle,...,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,196,49,M,writer,55105,242,3,881250949,242,Kolya (1996),...,0,0,0,0,0,0,0,0,0,0
1,186,39,F,executive,00000,302,3,891717742,302,L.A. Confidential (1997),...,0,1,0,0,1,0,0,1,0,0
2,22,25,M,writer,40206,377,1,878887116,377,Heavyweights (1994),...,0,0,0,0,0,0,0,0,0,0
3,244,28,M,technician,80525,51,2,880606923,51,Legends of the Fall (1994),...,0,0,0,0,0,1,0,0,1,1
4,166,47,M,educator,55113,346,1,886397596,346,Jackie Brown (1997),...,0,0,0,0,0,0,0,0,0,0


<b>Some data exploration:

In [5]:
#What are the 25 most rated movies?
most_rated = df.groupby('movietitle').size().sort_values(ascending=False)[:10]
most_rated

movietitle
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
dtype: int64

In [6]:
#Which movies are highly rated?
movie_stats = df.groupby('movietitle').agg({'rating': [np.size, np.mean]})
movie_stats.head()

rating          
                            size      mean
movietitle                                
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

In [7]:
#Sort by movie rating average
movie_stats.sort_values([('rating', 'mean')], ascending=False).head()

rating     
                                                    size mean
movietitle                                                   
Aiqing wansui (1994)                                   1  5.0
Prefontaine (1997)                                     3  5.0
Entertaining Angels: The Dorothy Day Story (1996)      1  5.0
Star Kid (1997)                                        3  5.0
Saint of Fort Washington, The (1993)                   2  5.0

In [8]:
#Movies that have been rated at least 100 times.
atleast_100 = movie_stats['rating']['size'] >= 100
movie_stats[atleast_100].sort_values([('rating', 'mean')], ascending=False)[:5]

rating          
                                   size      mean
movietitle                                       
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230

<b>Load 100K movie data as dictionary:

In [9]:
# Get movie titles
import urllib2
url = 'https://raw.githubusercontent.com/prcuny/DATA643/master/Project-2/'
movies={}
response = urllib2.urlopen(url + 'u.item')
for line in response:    
        (id,title)=line.split('|')[0:2]
        movies[id]=title


In [10]:
# Load user ratings
mdict={}
response = urllib2.urlopen(url + 'u.data')
for line in response:
       (user,movieid,rating,ts)=line.split('\t')
       mdict.setdefault(user,{})
       mdict[user][movies[movieid]]=float(rating)


Print first 2 users and their ratings

In [11]:
#user '20' and ratings
#mdict['20']
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n ))
# Dont know how to stop this from returning a bunch of rows
#take(2, mdict.iteritems())

### User-User Collaborative Filtering

<b> Pearson correlation

In [12]:
#Function that returns the Pearson correlation coefficient for person1 and person2 
def sim_pearson(dict,p1,p2):
 #Get the list of mutually rated items, ignore items that were not rated by either person
 si = {}
 for item in dict[p1]:
  if item in dict[p2] and not(np.isnan(dict[p2][item])) and not(np.isnan(dict[p1][item])): 
   si[item] = 1

 #if they are no rating in common, return 0
 if len(si) == 0:
  return 0

 #sum calculations
 n = len(si)
            
 #sum of all preferences
 sum1 = sum([dict[p1][it] for it in si])
 sum2 = sum([dict[p2][it] for it in si])
 #Sum of the squares
 sum1Sq = sum([pow(dict[p1][it],2) for it in si])
 sum2Sq = sum([pow(dict[p2][it],2) for it in si])
 #Sum of the products
 pSum = sum([dict[p1][it] * dict[p2][it] for it in si])
 #Calculate r (Pearson score)
 num = pSum - (sum1 * sum2)/n
 den = sqrt((sum1Sq - pow(sum1,2)/n) * (sum2Sq - pow(sum2,2)/n))
 if den == 0:
  return 0
 r = num/den
 return r


In [13]:
# Test the Pearson-based similarity function
round(sim_pearson(mdict,'20','30'), 4)

-0.4229

<b> Recommendations by Pearson weighted average: 

In [14]:
#Function that returns 'n' recommendations for a person by using a weighted average of every other user's rankings

def getRecommendations(n, dict,person,similarity=sim_pearson):
 totals = {}
 simSums = {}

 for other in dict:
  #don't compare me to myself
  if other == person:
   continue
  sim = similarity(dict,person,other)
  #ignore scores of zero or lower
  if sim <= 0: 
   continue
  for item in dict[other]:
   #only score movies I haven't rated yet
   if (item not in dict[person]) or (dict[person][item] == 0) or (np.isnan(dict[person][item])) :
    #Similarity * score
    totals.setdefault(item,0)
    totals[item] += not(np.isnan(dict[other][item] * sim))
    #Sum of similarities
    simSums.setdefault(item,0)
    simSums[item] += sim

    #Create the normalized list
 rankings = [(total/simSums[item],item) for item,total in totals.items()]

 #Return the sorted list
 rankings.sort()
 rankings.reverse()
 return rankings[0:n]


<b> Get top 10 recommendations for user '20' using User-User CF

In [15]:
#Test getRecommendations function
getRecommendations(10, mdict,'20',similarity=sim_pearson)

[(236.52695406660396, 'Golden Earrings (1947)'),
 (88.44403280479239, 'Johns (1996)'),
 (88.44403280479239, 'Beans of Egypt, Maine, The (1994)'),
 (87.25594535617968, 'Paradise Road (1997)'),
 (55.054047052742746, 'Cosi (1996)'),
 (55.054047052742746, 'All Over Me (1997)'),
 (53.94433001475551, 'Favor, The (1994)'),
 (49.580099539706296,
  'Brother Minister: The Assassination of Malcolm X (1994)'),
 (37.8437564385548, 'You So Crazy (1994)'),
 (37.8437564385548, 'Walking Dead, The (1995)')]

### Code to implement user/user, item/item and user/item using pandas

In [16]:
# function takes in a data frame, what you want to predict pred_id, the type of collaboration, the similarity you want
# to use, and the number of responses you want to get back
## Code adapted from https://blog.liip.ch/archive/2016/06/16/recommender-system-slack-pandas-flask.html

def predict_p(dframe, pred_id, kind = 'user', similarity = cosine_similarity, n = 10):
    d = dframe.fillna(0)
    if kind == 'user':
        dists = similarity(d)
        dists_named = pd.DataFrame(dists, columns = d.index)
        dists_named.index = dists_named.columns
        
    elif kind == 'item':
        d = d.T
        dists = similarity(d)
        dists_named = pd.DataFrame(dists, columns = d.index)
        dists_named.index = dists_named.columns

    p = dists_named[pred_id].sort_values(ascending = False)
    mask = p.index.isin([pred_id])
    p = p[~mask]
    return p[:n]   

In [17]:
# convert to wide format for analysis
d = df.pivot_table(index = 'userid', columns = 'movietitle', values = 'rating')
d = d.fillna(0)
del d.index.name
d.head()

movietitle,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0


In [18]:
# Predict based on item/item
predict_p(d, pred_id = "Amadeus (1984)", kind = 'item')

movietitle
Forrest Gump (1994)                       0.661877
Raiders of the Lost Ark (1981)            0.660880
Back to the Future (1985)                 0.643994
E.T. the Extra-Terrestrial (1982)         0.637103
Dead Poets Society (1989)                 0.636788
One Flew Over the Cuckoo's Nest (1975)    0.636475
Silence of the Lambs, The (1991)          0.628871
2001: A Space Odyssey (1968)              0.627704
When Harry Met Sally... (1989)            0.627460
Schindler's List (1993)                   0.617628
Name: Amadeus (1984), dtype: float64

In [37]:
# Predict based on user/user
## At present this gives the closest users
predict_p(d, pred_id = 290, kind = 'user')

254    0.575197
738    0.568915
881    0.565369
42     0.547329
545    0.543806
804    0.543751
650    0.543560
275    0.540358
815    0.536989
536    0.532302
Name: 290, dtype: float64

In [35]:
# Recommend items based on User similarity
def get_recommendations_user(dframe, pred_id, similarity = cosine_similarity, n = 10):
    t = d.ix[predict_p(d, pred_id = pred_id, n = n).index]
    #t.ix[:,:10]
    p = d.ix[pred_id].values != 0



    r = t.T.ix[~p].T
    # Delete those columns that all contain all 0, meaning that they have not been rated
    r = r.loc[:, (r != 0).any(axis=0)]
    # Have to rename the columns as we keep on getting an error
    cols = r.columns
    r.columns =  range(1, 1+ r.shape[1])
    # Calculate the mean ratings for the movies
    ratings = r.sum(axis = 0)/r.astype(bool).sum(axis= 0)
    ratings.index = cols
    return ratings.sort_values(axis = 0, ascending = False, 
                inplace = False, na_position = 'last')[:n]
    

In [36]:
get_recommendations_user(d, pred_id = 290, similarity = cosine_similarity, n = 10)

movietitle
Enchanted April (1991)          5.0
Ran (1985)                      5.0
Perfect World, A (1993)         5.0
Persuasion (1995)               5.0
Little Princess, A (1995)       5.0
My Man Godfrey (1936)           5.0
Shadowlands (1993)              5.0
Sense and Sensibility (1995)    5.0
Army of Darkness (1993)         5.0
That Thing You Do! (1996)       5.0
dtype: float64

In [39]:
# Check the results
cut = [254, 738,881, 42,545,804,650,275,815,536]
d.loc[cut,'Enchanted April (1991)']


254    0.0
738    0.0
881    0.0
42     0.0
545    0.0
804    0.0
650    0.0
275    0.0
815    0.0
536    5.0
Name: Enchanted April (1991), dtype: float64

<b>Split the dataset into training and testing sets

In [23]:
from sklearn import cross_validation as cv
df1 = df[["userid", "itemid", "rating"]]
df1.columns = ["userid", "itemid", "rating"]
train_data, test_data = cv.train_test_split(df1, test_size=0.25)

In [24]:
n_users = df1.userid.unique().shape[0]
n_items = df1.itemid.unique().shape[0]
print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)  

Number of users = 943 | Number of movies = 1682


In [25]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [26]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')


In [27]:
item_similarity

array([[  0.00000000e+00,   7.20705709e-01,   7.01668384e-01, ...,
          1.00000000e+00,   1.00000000e+00,   9.46504747e-01],
       [  7.20705709e-01,   3.33066907e-16,   7.96730504e-01, ...,
          1.00000000e+00,   1.00000000e+00,   9.02564530e-01],
       [  7.01668384e-01,   7.96730504e-01,   0.00000000e+00, ...,
          1.00000000e+00,   1.00000000e+00,   8.95107460e-01],
       ..., 
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  9.46504747e-01,   9.02564530e-01,   8.95107460e-01, ...,
          1.00000000e+00,   1.00000000e+00,   0.00000000e+00]])

In [28]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [29]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [30]:
user_prediction

array([[ 1.64063357,  0.54971107,  0.5185093 , ...,  0.31010091,
         0.30755236,  0.30964338],
       [ 1.36196382,  0.23176473,  0.16551721, ..., -0.0734258 ,
        -0.07521922, -0.07189619],
       [ 1.37811308,  0.19491418,  0.14071789, ..., -0.10685359,
        -0.10842962, -0.10502381],
       ..., 
       [ 1.23195309,  0.15959259,  0.10220474, ..., -0.12742319,
        -0.12973088, -0.12648355],
       [ 1.41870002,  0.26645167,  0.22625882, ..., -0.01446737,
        -0.01659307, -0.01345373],
       [ 1.44073657,  0.33798803,  0.30662695, ...,  0.0953558 ,
         0.09282906,  0.0953446 ]])

<b>Evaluation

In [31]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

In [32]:
print 'User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))
print 'Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix))

User-based CF RMSE: 3.13126405047
Item-based CF RMSE: 3.45734584911
